# Retrieval-augmented generation (RAG)


In [1]:
!pip install langchain==0.0.331 langchainhub 

In [2]:
!pip install openai==0.28.1

In [3]:
!pip install faiss-cpu tiktoken

  Using cached tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (773 kB)


In [4]:
!pip install python-dotenv 

In [5]:
!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.11.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached scipy-1.11.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.4 MB)
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


In [6]:
ls -la

total 28
drwxrwxr-x 3 wasit wasit  4096 Nov  8 14:22 ./
drwxrwxr-x 5 wasit wasit  4096 Nov  7 10:46 ../
-rw-rw-r-- 1 wasit wasit    66 Nov  7 10:57 .env
drwxrwxr-x 2 wasit wasit  4096 Nov  7 10:48 .ipynb_checkpoints/
-rw-rw-r-- 1 wasit wasit 10668 Nov  8 14:22 RAG.ipynb


In [7]:
# Set env var OPENAI_API_KEY or load from a .env file
import dotenv

dotenv.load_dotenv()

True

In [8]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [16]:
loader = WebBaseLoader("https://www.history.com/topics/ancient-rome/ancient-rome")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(loader.load())

In [17]:
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# vectorstore.save_local('vector.db')
# vectorstore=FAISS.load_local('vector.db', embeddings=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [18]:
# from langchain.retrievers import SVMRetriever
# question="What are key facts from the articles?"
# svm_retriever = SVMRetriever.from_documents(splits, OpenAIEmbeddings())
# docs_svm = svm_retriever.get_relevant_documents(question)
# len(docs_svm)

In [19]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | rag_prompt_custom | llm
)

In [20]:
print(rag_chain.invoke("What are key facts from the articles?").content)

The Roman Empire was founded in 27 B.C. and was a vast and powerful domain. It gave rise to the culture, laws, technologies, and institutions that continue to define Western civilization. The articles were written or edited by the HISTORY.com editors, including Amanda Onion, Missy Sullivan, Matt Mullen, and Christian Zapata. The articles provide information about Ancient Rome, including its location and timeline. The content is regularly reviewed and updated for accuracy and fairness.


In [23]:
print(rag_chain.invoke("Why did the Roman empire fell?").content)

The Roman Empire fell due to a combination of factors including constant warfare, invasions by barbarian tribes, internal turmoil and political instability, widening wealth inequality, and the loss of provinces.


In [15]:
# from langchain.llms import GPT4All
# local_path = (
#     "./models/ggml-gpt4all-l13b-snoozy.bin"  # replace with your desired local file path
# )

# # Callbacks support token-wise streaming
# callbacks = [StreamingStdOutCallbackHandler()]

# # Verbose is required to pass to the callback manager
# llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)